In [1]:
import arcpy
from arcpy.sa import *
import os
import time
arcpy.env.overwriteOutput = True

In [2]:
version='v13_241021'
sub_v=''
# stastic_v=f'{version}_{sub_v}'
stastic_v=f'{version}'
output_dir = os.path.join(r'J:\lakemapping\postprocess',version)

auxiliary_dataset_gdb=r'D:\postprocess\v240220\auxiliary_dataset.gdb'
mwBG_gdb=os.path.join(output_dir,f'5_polygon_afm_mwBL_BigGLAKES{sub_v}.gdb')
temp_gdb=os.path.join(output_dir,'7_correct_temp_file.gdb')
if not os.path.exists(temp_gdb):
    arcpy.CreateFileGDB_management(output_dir,f'7_correct_temp_file.gdb')
mwBG_point_gdb=os.path.join(output_dir,f'5_polygon_afm_mwBL_BigGLAKES{sub_v}_point.gdb')

In [3]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateField(inputFeature,field,code):
    arcpy.CalculateField_management(inputFeature, field,code)

def cal_continent(lyr):
    arcpy.AddField_management(lyr, 'Continent', "Text")
    for i in range(0,6):
        continent=seven_continents[i]
        selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
        selectByAttribute(lyr,'NEW_SELECTION',"Continent IS NULL")
        selectByLocation(lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
        calculateField(lyr,'Continent',f"'{continent}'")

In [4]:
grid=os.path.join(temp_gdb,'Global_grid_shp_1degree')
grid_lyr='Global_grid_shp_1degree'
arcpy.MakeFeatureLayer_management(grid,grid_lyr)

<Result 'Global_grid_shp_1degree'>

In [6]:
eight_continents=['Asia','Siberia','Africa','Europe','Oceania_Anta','North_America','Arctic','South_America']

In [ ]:
merge_list=[]  
for i in range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_BL_BG_point=os.path.join(mwBG_point_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG_point')
    c_lake_afm_mw_BL_BG_point_lyr=f'c{i+1}_{continent}_polygon_afm_mwBL_BG_point'
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG_point,c_lake_afm_mw_BL_BG_point_lyr)
    merge_list.append(c_lake_afm_mw_BL_BG_point_lyr)

for i in range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_BL_BG_point_lyr=f'c{i+1}_{continent}_polygon_afm_mwBL_BG_point'
    selectByAttribute(c_lake_afm_mw_BL_BG_point_lyr,'NEW_SELECTION','lake_type = 1')
global_lake_afm_mw_BL_BG_small_point=os.path.join(mwBG_point_gdb,'global_polygon_afm_mwBL_BG_point_small_lake')
arcpy.Merge_management(merge_list,global_lake_afm_mw_BL_BG_small_point)

for i in range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_BL_BG_point_lyr=f'c{i+1}_{continent}_polygon_afm_mwBL_BG_point'
    selectByAttribute(c_lake_afm_mw_BL_BG_point_lyr,'NEW_SELECTION','lake_type = 2')
global_lake_afm_mw_BL_BG_point=os.path.join(mwBG_point_gdb,'global_polygon_afm_mwBL_BG_point_median_lake')
arcpy.Merge_management(merge_list,global_lake_afm_mw_BL_BG_point)

for i in range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_BL_BG_point_lyr=f'c{i+1}_{continent}_polygon_afm_mwBL_BG_point'
    selectByAttribute(c_lake_afm_mw_BL_BG_point_lyr,'NEW_SELECTION','lake_type = 3')
global_lake_afm_mw_BL_BG_point=os.path.join(mwBG_point_gdb,'global_polygon_afm_mwBL_BG_point_big_lake')
arcpy.Merge_management(merge_list,global_lake_afm_mw_BL_BG_point)

In [9]:
for i in range(6,7):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_BL_BG=os.path.join(mwBG_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
    c_lake_afm_mw_BL_BG_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG'
    c_lake_afm_mw_BL_BG_pi=os.path.join(temp_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG_pair_inter')
    c_lake_afm_mw_BL_BG_pi_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG_pi'
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG,c_lake_afm_mw_BL_BG_lyr)
#     c_lake_afm_mw_BL_BG_point=os.path.join(mwBG_point_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG_point')
#     arcpy.management.FeatureToPoint(c_lake_afm_mw_BL_BG_lyr,c_lake_afm_mw_BL_BG_point,'INSIDE')
    arcpy.analysis.PairwiseIntersect([c_lake_afm_mw_BL_BG_lyr,grid_lyr],c_lake_afm_mw_BL_BG_pi,'NO_FID')
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG_pi,c_lake_afm_mw_BL_BG_pi_lyr)
    selectByLocation(c_lake_afm_mw_BL_BG_pi_lyr,'SHARE_A_LINE_SEGMENT_WITH',grid_lyr,'NEW_SELECTION')
    arcpy.CalculateField_management(c_lake_afm_mw_BL_BG_pi_lyr,'lake_area', "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

    
    c_lake_afm_mw_BL_BG_pi_point=os.path.join(mwBG_point_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG_pi_point')
    selectByAttribute(c_lake_afm_mw_BL_BG_pi_lyr,'NEW_SELECTION','')
    arcpy.management.FeatureToPoint(c_lake_afm_mw_BL_BG_pi_lyr,c_lake_afm_mw_BL_BG_pi_point,'INSIDE')

Arctic


In [3]:
PLD_gdb=r'D:\lakemapping\9_analysis\analysis.gdb'
new_PLD_gdb=r'J:\lakemapping\PLD\PLD.gdb'
PLD_point_gdb=r'J:\lakemapping\PLD\PLD_point.gdb'
PLD_grid_gdb=r'J:\lakemapping\PLD\PLD_grid.gdb'
if not os.path.exists(PLD_point_gdb):
    arcpy.CreateFileGDB_management(r'J:\lakemapping\PLD',f'PLD_point.gdb')
if not os.path.exists(PLD_grid_gdb):
    arcpy.CreateFileGDB_management(r'J:\lakemapping\PLD',f'PLD_grid.gdb')
arcpy.env.workspace = PLD_gdb
PLD = arcpy.ListFeatureClasses()
print(PLD)

['different_area', 'PLD_eu_21', 'PLD_eu_29', 'PLD_eu_22', 'PLD_eu_28', 'PLD_eu_24', 'PLD_eu_25', 'PLD_eu_23', 'PLD_eu_27', 'PLD_eu_26', 'PLD_as_45', 'PLD_as_49', 'PLD_as_44', 'PLD_as_43', 'PLD_as_47', 'PLD_as_46', 'PLD_as_42', 'PLD_as_41', 'PLD_si_31', 'PLD_si_32', 'PLD_si_33', 'PLD_si_34', 'PLD_si_35', 'PLD_si_36', 'PLD_au_51', 'PLD_au_52', 'PLD_au_53', 'PLD_na_73', 'PLD_na_74', 'PLD_na_75', 'PLD_na_76', 'PLD_na_77', 'PLD_ar_81', 'PLD_ar_86', 'PLD_gr_91', 'PLD_ar_84', 'PLD_ar_82', 'PLD_na_78', 'PLD_af_14', 'PLD_af_11', 'PLD_af_12', 'PLD_af_13', 'PLD_af_15', 'PLD_af_16', 'PLD_af_17', 'PLD_af_18', 'PLD_au_56', 'PLD_au_54', 'PLD_au_55', 'PLD_au_57', 'PLD_sa_61', 'PLD_sa_62', 'PLD_sa_63', 'PLD_sa_65', 'PLD_sa_66', 'PLD_sa_67', 'PLD_sa_64', 'PLD_ar_85', 'PLD_na_72', 'PLD_ar_83', 'PLD_na_71', 'PLD_as_48', 'PLD_eu_22_point', 'PLD_eu_28_point', 'PLD_eu_24_point', 'PLD_eu_25_point', 'PLD_eu_23_point', 'PLD_eu_27_point', 'PLD_eu_26_point', 'PLD_as_45_point', 'PLD_as_49_point', 'PLD_as_44_point'

In [27]:
for layer in PLD[1:2]:
    PLD_data=os.path.join(PLD_gdb,layer)
    PLD_data_layer='PLD'
    arcpy.MakeFeatureLayer_management(PLD_data,PLD_data_layer)
    PLD_data_point=os.path.join(PLD_gdb,layer+'_point')
#     arcpy.management.FeatureToPoint(PLD_data_layer,PLD_data_point,'INSIDE')
    PLD_data_pi=os.path.join(PLD_grid_gdb,layer+'_pairwise_intersect')
#     arcpy.analysis.PairwiseIntersect([PLD_data_layer, grid_lyr],PLD_data_pi,join_attributes='NO_FID')
    PLD_data_pi_lyr=f'{layer}_data_pi'
    arcpy.MakeFeatureLayer_management(PLD_data_pi,PLD_data_pi_lyr)
    selectByLocation(PLD_data_pi_lyr,'SHARE_A_LINE_SEGMENT_WITH',grid_lyr,'NEW_SELECTION')
    arcpy.CalculateField_management(PLD_data_pi_lyr,'ref_area', "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
    PLD_data_pi_point=os.path.join(PLD_point_gdb,layer+'_pi_point')
    selectByAttribute(PLD_data_pi_lyr,"NEW_SELECTION","")
    arcpy.management.FeatureToPoint(PLD_data_pi_lyr,PLD_data_pi_point,'INSIDE')

In [31]:
PLD_pi_point=os.path.join(PLD_point_gdb,'total_PLD_pi_point')
PLD_SJ=os.path.join(PLD_gdb,'PLD_SJ_1')
arcpy.MakeFeatureLayer_management(PLD_pi_point,'PLD_pi_point')
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable('PLD_pi_point')

field_idx = fieldmappings_all.findFieldMapIndex('ref_area')
fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))

area_idx = fieldmappings.findFieldMapIndex("ref_area")
area_fieldmap = fieldmappings.getFieldMap(area_idx)
area_fieldmap.mergeRule = "sum"
fieldmappings.replaceFieldMap(area_idx, area_fieldmap)

arcpy.SpatialJoin_analysis(grid_lyr,'PLD_pi_point',PLD_SJ,"JOIN_ONE_TO_ONE","KEEP_ALL", fieldmappings, "CONTAINS")

<Result 'D:\\lakemapping\\9_analysis\\analysis.gdb\\PLD_SJ_1'>

In [35]:
arcpy.MakeFeatureLayer_management(PLD_SJ,'PLD_SJ')
PLD_point=os.path.join(PLD_point_gdb,'total_PLD_point')
PLD_SJ_2=os.path.join(PLD_gdb,'PLD_SJ_2')
arcpy.MakeFeatureLayer_management(PLD_point,'PLD_point')

fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()

fieldmappings_all.addTable('PLD_SJ')
field_idx = fieldmappings_all.findFieldMapIndex('ref_area')
fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))

fieldmappings_all.addTable('PLD_point')
field_idx = fieldmappings_all.findFieldMapIndex('lake_lt3ha')
fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))


field_idx = fieldmappings.findFieldMapIndex("lake_lt3ha")
field_fieldmap = fieldmappings.getFieldMap(field_idx)
field_fieldmap.mergeRule = "sum"
fieldmappings.replaceFieldMap(field_idx, field_fieldmap)

arcpy.SpatialJoin_analysis('PLD_SJ','PLD_point',PLD_SJ_2,"JOIN_ONE_TO_ONE","KEEP_ALL", fieldmappings, "CONTAINS")

<Result 'D:\\lakemapping\\9_analysis\\analysis.gdb\\PLD_SJ_2'>

In [36]:
arcpy.MakeFeatureLayer_management(PLD_point,'PLD_point')

<Result 'PLD_point'>

In [5]:
PLD_data=os.path.join(PLD_gdb,'PLD_eu_29')
PLD_data_layer='PLD_eu_29'
arcpy.MakeFeatureLayer_management(PLD_data,PLD_data_layer)

field_list=['basin_id','ref_area','res_id','sources']
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(PLD_data_layer)#未mask
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
        
for region in ['af','eu','si','as','au','sa','na','ar','gr']:
    merge_list=[]
    PLD_region=os.path.join(new_PLD_gdb,f'PLD_{region}')
    for layer in PLD:
        if layer.startswith(f'PLD_{region}') and not layer.endswith('point'):
            layer_dir=os.path.join(PLD_gdb,layer)
            merge_list.append(layer_dir)
    arcpy.Merge_management(merge_list,PLD_region,field_mappings=fieldmappings)

# merge_list=[]
# PLD_Rser=os.path.join(new_PLD_gdb,f'PLD_Rser')
# for region in ['af','eu','si','as','au','sa','na','ar','gr']:
#     for layer in PLD:
#         if layer.startswith(f'PLD_{region}') and not layer.endswith('point'):
#             layer_dir=os.path.join(PLD_gdb,layer)
#             merge_list.append(layer_dir)
# arcpy.Merge_management(merge_list,PLD_Rser,field_mappings=fieldmappings)

## my result

In [ ]:
global_lake_afm_mw_BL_BG_small_point=os.path.join(mwBG_point_gdb,'global_polygon_afm_mwBL_BG_point_small_lake')
global_lake_afm_mw_BL_BG_small_point_lyr='global_lake_afm_mw_BL_BG_small_point'
arcpy.MakeFeatureLayer_management(global_lake_afm_mw_BL_BG_small_point,global_lake_afm_mw_BL_BG_small_point_lyr)
#解决GLAKES 八连通湖泊 不一致的问题
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(global_lake_afm_mw_BL_BG_small_point_lyr)

# field_idx = fieldmappings_all.findFieldMapIndex('JoinCount')
# fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))

selectByAttribute(global_lake_afm_mw_BL_BG_small_point_lyr,"NEW_SELECTION","lake_area < 0.01")
print('Conduct spatial join')
arcpy.SpatialJoin_analysis(grid_lyr, global_lake_afm_mw_BL_BG_small_point_lyr, b_iwBG_merge_with_BG_SJ_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "LARGEST_OVERLAP")

In [3]:
auxiliary_dataset_gdb=r'D:\postprocess\v240220\auxiliary_dataset.gdb'
oceanline_polygon=os.path.join(auxiliary_dataset_gdb,'oceanline_polygon')
arcpy.MakeFeatureLayer_management(oceanline_polygon,'oceanline_polygon')

<Result 'oceanline_polygon'>

# 恢复部分连接海洋的湖泊

In [4]:
raw_prediction_dir=r'J:\lakemapping\postprocess\v2_240220\0_raw_prediction.gdb'
lake_intersect_ocean=r'J:\lakemapping\postprocess\v13_241021\supplement_lake.gdb\lake_intersect_ocean'
arcpy.env.workspace = raw_prediction_dir
raw_prediction_list = arcpy.ListFeatureClasses()

start=time.time()
for raw_prediction_fn in raw_prediction_list:#,'lakes_eu','lakes_as_7'
    print(raw_prediction_fn)
    raw_prediction = os.path.join(raw_prediction_dir,raw_prediction_fn)#version dissolve geometry shp:copy_sldissolve.shp
    raw_prediction_layer=raw_prediction_fn
    arcpy.MakeFeatureLayer_management(raw_prediction, raw_prediction_layer)
    arcpy.SelectLayerByLocation_management(raw_prediction_layer, 'INTERSECT', oceanline_polygon)

lakes_northern_america_1_dissolved
lakes_northern_america_3_dissolved
lakes_northern_america_4_dissolved
lakes_southern_america_2_dissolved
lakes_europe_1_dissolved
lakes_europe_2_dissolved
lakes_africa_1_dissolved
lakes_africa_3_dissolved
lakes_asia_1_dissolved
lakes_asia_3_dissolved
lakes_asia_4_dissolved
lakes_asia_2
lakes_asia_7
lakes_middle_east
lakes_africa_2
lakes_northern_america_2
lakes_northern_america_5
lakes_southern_america_1
lakes_island
lakes_greenland
lakes_Australia
lakes_asia_5_dissolved
lakes_Japan


In [5]:
arcpy.Merge_management(raw_prediction_list,lake_intersect_ocean)
arcpy.MakeFeatureLayer_management(lake_intersect_ocean,'lake_intersect_ocean')
print('calculate area')
arcpy.AddField_management('lake_intersect_ocean', 'area', "Double")########################
arcpy.CalculateField_management('lake_intersect_ocean', "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
arcpy.management.SelectLayerByAttribute('lake_intersect_ocean','NEW_SELECTION', '"area" < 0.005')


calculate area


<Result 'lake_intersect_ocean'>

In [7]:
arcpy.DeleteFeatures_management('lake_intersect_ocean')

<Result 'lake_intersect_ocean'>

In [47]:
for region in ['af','eu','si','as','au','sa','na','ar','gr']:
    PLD_lyr=f'PLD_{region}'
    arcpy.SelectLayerByLocation_management('b_iwBG_merge_with_BG_final', 'INTERSECT', PLD_lyr)
    arcpy.CalculateField_management('b_iwBG_merge_with_BG_final','iwPLD',1)